In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR
from src.database import DB
from src.instance.Instance import Instance
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

In [2]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

X = np.random.uniform(size=(200, 137))
y = np.random.uniform(size=(200,))
model.fit(X, y)

RandomForestRegressor()

In [2]:
train_instances, test_instances = TSP_train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=3,
)
train_instances

InstanceList(size=3)[TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp)]

In [3]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=2)
portfolio

Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]

In [4]:
result = portfolio.evaluate(
    train_instances,
    "xd2",
    calculate_features=True,
    cache=False,
    estimator=None,
)

[2024-12-25 23:06:09] DEBUG     Portfolio.evaluate(xd2)
[2024-12-25 23:06:09] DEBUG     Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]
[2024-12-25 23:06:09] DEBUG     solve(prefix=xd2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-25 23:06:09] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-25 23:06:15] DEBUG     solve(prefix=xd2, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-25 23:06:15] DEBUG     solve(prefix=xd2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp))
[2024-12-25 23:06:15] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp))
[2024-12-25 23:06:18] DEBUG     solve(prefix=xd2, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEP

In [6]:
db = DB()

In [7]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,...,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,855157651668393360,TSP/CEPS_benchmark/explosion/21.tsp,14928292.0,0.006290,1.484946,1.522267,3.139028,0.916031,3.132738,0.601754,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1353678304570487518,TSP/CEPS_benchmark/rotation/34.tsp,15486460.0,0.000684,1.495912,1.523094,3.138384,0.912371,3.137700,0.599025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,810870798045289410,TSP/CEPS_benchmark/compression/09.tsp,16003060.0,0.000000,1.417332,1.483901,3.141593,1.026220,3.141593,0.691569,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,473831520656465062,0.25,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.5,...,0.470388,0.888889,1.00,0.6,0.2,0.244898,0.0,1.0,3.0,1.0


In [9]:
pd.read_sql_query("SELECT * FROM results", db._conn)

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,xd2_473831520656465062_855157651668393360,xd2,473831520656465062,855157651668393360,100.0,10.000000,0,0,0
1,xd2_473831520656465062_1353678304570487518,xd2,473831520656465062,1353678304570487518,100.0,10.000000,0,0,0
2,xd2_52980316141715150_1353678304570487518,xd2,52980316141715150,1353678304570487518,100.0,13.125985,0,0,0
3,xd2_52980316141715150_855157651668393360,xd2,52980316141715150,855157651668393360,100.0,15.780565,0,0,0
4,xd2_473831520656465062_810870798045289410,xd2,473831520656465062,810870798045289410,100.0,10.000000,0,0,0
5,xd2_52980316141715150_810870798045289410,xd2,52980316141715150,810870798045289410,100.0,10.000000,0,0,1


In [37]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)

,id,solver_id,instance_id,cost
0,52980316141715150_1353678304570487518,52980316141715150,1353678304570487518,100.0
1,473831520656465062_1353678304570487518,473831520656465062,1353678304570487518,100.0
2,473831520656465062_855157651668393360,473831520656465062,855157651668393360,100.0
3,52980316141715150_855157651668393360,52980316141715150,855157651668393360,100.0
4,473831520656465062_810870798045289410,473831520656465062,810870798045289410,100.0
5,52980316141715150_810870798045289410,52980316141715150,810870798045289410,100.0


In [49]:
import sys
import pickle

my_object = portfolio
in_memory_size = sys.getsizeof(my_object)
serialized_object = pickle.dumps(my_object)
serialized_size = len(serialized_object)

print(f"In-memory size: {in_memory_size} bytes")
print(f"Serialized size: {serialized_size} bytes")

In-memory size: 104 bytes
Serialized size: 19124 bytes
